In [ ]:
import os
import sys
from pathlib import Path
import datetime
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.merge import merge
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches

#import shutil
#import tempfile
#import json
#import random
import numpy as np
import pandas as pd
#import geopandas as gpd
#from pyproj import Proj, transform
#from pyproj import CRS
#from shapely.geometry import box
#from shapely.geometry import shape
#from shapely.geometry import MultiPoint
#from shapely.geometry import Point
#from shapely.geometry import Polygon
#import xarray as xr

%matplotlib inline

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n gridCell = {} \n index_dir = {} \n out_dir = {}"
      .format(basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir']))

%store -r SingleOutputParams
print("SingleOutputParams: \n ClassPrefix = {} \n ClassFile = {}"
      .format(SingleOutputParams['ClassPrefix'],SingleOutputParams['ClassFile']))

## Define classification system

In [ ]:
##TODO: Make dictionaries for each classification option
LC_labels = ["bare/built",
            "water",
            "lowveg",
            "medVeg",
            "plantation",
            "mature forest"]
colors = ['linen',
          'blue',
          'lightgreen',
          'yellowgreen',
          'purple',
          'darkgreen'
         ]
cmap = ListedColormap(colors)

patches = []
for lc in range(len(LC_labels)):
    patch = mpatches.Patch(color=colors[lc], label=LC_labels[lc])
    patches.append(patch)

class_bins = [1, 7, 10, 40, 60, 80,100]
norm = BoundaryNorm(class_bins,
                    len(colors))

## Get band info

In [ ]:
with rasterio.open(SingleOutputParams['ClassFile']) as src:
    Stack = src.read()
    NumBands = Stack.shape[0]
    
##Get band years from input data (assumes that data in selected index directory corresponds to that used in classification)
##TODO: make option of inputing own start year instead.

YrStack = []
print("Getting year list from: {}".format(basicConfig['index_dir']))
for r in os.listdir(basicConfig['index_dir']):
    if r.endswith('.tif'):
        InYr = r[:4]
        if InYr not in YrStack:
            YrStack.append(InYr)
YrStack.sort()
##remove the first element in the list (this is the padding and probably not in the final; heck if this is always the case)
print("skipping first year in TS (used as padding)")
YrStack.pop(0)
StackData = pd.DataFrame({'MapYear':YrStack})
StackData.index.name='Band'
print('Bands in classified output are:')
print(StackData.T)

## plot classified map at multiple timepoints

In [ ]:
TotPlots = len(SingleOutputParams['MapYears'])
Cols = 2
Rows = TotPlots // Cols 
Rows += TotPlots % Cols
Position = range(1,TotPlots + 1)

fig = plt.figure(1, figsize=(20,20))
fig.legend(handles=patches, loc='center')

for k in range(TotPlots):
    ax = fig.add_subplot(Rows,Cols,Position[k])
    with rasterio.open(SingleOutputParams['ClassFile']) as src:
        ClassYr = src.read(k+1)
    ax.imshow(ClassYr, cmap=cmap, norm=norm)
    ax.set_title("Year {}".format(SingleOutputParams['MapYears'][k]))
    ax.axis('off')

## view change in plantations over time

## view change in natural forest over time

## mosaic multiple cells

In [ ]:
GridCells = [1157,1158]  #TODO: Make this a parameter
src_to_mosaic = []
for g in GridCells:
    ClassFile = os.path.join(basicConfig['smooth_dir'],'{:06d}'.format(g),'cls',SingleOutputParams['ClassPrefix']+'{:06d}'.format(g)+'.tif')
    src = rasterio.open(ClassFile)
    src_to_mosaic.append(src[1])
mosaic, out_trans = merge(src_to_mosaic)
show(mosaic, cmap=cmap)

## view alongside actual image